# Введение в обработку текста на естественном языке

Материалы:
* Макрушин С.В. Лекция 9: Введение в обработку текста на естественном языке\
* https://realpython.com/nltk-nlp-python/
* https://scikit-learn.org/stable/modules/feature_extraction.html

## Задачи для совместного разбора

In [42]:
from sklearn.feature_extraction.text import CountVectorizer
import pymorphy2

1. Считайте слова из файла `litw-win.txt` и запишите их в список `words`. В заданном предложении исправьте все опечатки, заменив слова с опечатками на ближайшие (в смысле расстояния Левенштейна) к ним слова из списка `words`. Считайте, что в слове есть опечатка, если данное слово не содержится в списке `words`. 

In [ ]:
text = '''с велечайшим усилием выбравшись из потока убегающих людей Кутузов со свитой уменьшевшейся вдвое поехал на звуки выстрелов русских орудий'''

2. Разбейте текст из формулировки задания 1 на слова; проведите стемминг и лемматизацию слов.

3. Преобразуйте предложения из формулировки задания 1 в векторы при помощи `CountVectorizer`.

## Лабораторная работа 9

### Расстояние редактирования

1.1 Загрузите предобработанные описания рецептов из файла `preprocessed_descriptions.csv`. Получите набор уникальных слов `words`, содержащихся в текстах описаний рецептов (воспользуйтесь `word_tokenize` из `nltk`). 

In [1]:
import pandas as pd
from nltk.tokenize import word_tokenize

In [2]:
import string

data = pd.read_csv('preprocessed_descriptions.csv')
description = data.preprocessed_descriptions.astype(str)

words = []
for i in description:
    # Приводим к нижнему регистру и удаляем знаки пунктуации
    i = i.lower()
    i = ''.join([char for char in i if char not in string.punctuation])
    
    words.extend(word_tokenize(i))

unique_words = list(set(words))
len(unique_words)

24485

1.2 Сгенерируйте 5 пар случайно выбранных слов и посчитайте между ними расстояние редактирования.

In [3]:
from nltk.metrics.distance import (
    edit_distance,
    edit_distance_align,
    binary_distance,
    jaccard_distance,
    masi_distance,
    interval_distance,
    custom_distance,
    presence,
    fractional_presence,
)

In [4]:
import random

random_words = random.sample(unique_words, 10)
for i in range(0, len(random_words)-3, 2):
    distance = edit_distance(random_words[i], random_words[i+1])
    print(f'Расстояние между словами {random_words[i]} и {random_words[i+1]}: {distance}')

Расстояние между словами tougher и silk: 7
Расстояние между словами timey и upto: 5
Расстояние между словами cattlemen и syria: 9
Расстояние между словами apatite и definately: 8


1.3 Напишите функцию, которая для заданного слова `word` возвращает `k` ближайших к нему слов из списка `words` (близость слов измеряется с помощью расстояния Левенштейна)

In [5]:
def nearest_words(word, k):
    dictance_dict = {w:edit_distance(word, w) for w in words}
    sorted_dict = dict(sorted(dictance_dict.items(), key=lambda x: x[1]))
    for key in list(sorted_dict.keys())[:k]:
        print(f'Расстояние от {word} до {key} равно {sorted_dict[key]}')

In [6]:
words = ['rfeeodmm', 'fereedom', 'fredeom', 'fredom', 
         'fredoom', 'frrteddmon']

nearest_words('freedom', 3)

Расстояние от freedom до fereedom равно 1
Расстояние от freedom до fredom равно 1
Расстояние от freedom до fredeom равно 2


### Стемминг, лемматизация

2.1 На основе результатов 1.1 создайте `pd.DataFrame` со столбцами: 
    * word
    * stemmed_word 
    * normalized_word 

Столбец `word` укажите в качестве индекса. 

Для стемминга воспользуйтесь `SnowballStemmer`, для нормализации слов - `WordNetLemmatizer`. Сравните результаты стемминга и лемматизации.

In [7]:
from nltk.stem import SnowballStemmer
from nltk.stem import WordNetLemmatizer

# Инициализация SnowballStemmer и WordNetLemmatizer
snb_stemmer = SnowballStemmer('english')
lemmatizer = WordNetLemmatizer()

In [8]:
stemmed_word = [snb_stemmer.stem(word) for word in unique_words]
normalized_word = [lemmatizer.lemmatize(word) for word in unique_words]

# Создание DataFrame с одинаковой длиной данных и индекса
data = {'stemmed_word': stemmed_word, 'normalized_word': normalized_word}
words_df = pd.DataFrame(data, index=unique_words)
words_df.index.name = 'word'

words_df

,stemmed_word,normalized_word
word,,
skyr,skyr,skyr
hindi,hindi,hindi
thewebgangsta,thewebgangsta,thewebgangsta
indefinite,indefinit,indefinite
shakamak,shakamak,shakamak
...,...,...
hunk,hunk,hunk
judy,judi,judy
laurien,laurien,laurien


2.2. Удалите стоп-слова из описаний рецептов. Какую долю об общего количества слов составляли стоп-слова? Сравните топ-10 самых часто употребляемых слов до и после удаления стоп-слов.

### Векторное представление текста

3.1 Выберите случайным образом 5 рецептов из набора данных. Представьте описание каждого рецепта в виде числового вектора при помощи `TfidfVectorizer`

3.2 Вычислите близость между каждой парой рецептов, выбранных в задании 3.1, используя косинусное расстояние (`scipy.spatial.distance.cosine`) Результаты оформите в виде таблицы `pd.DataFrame`. В качестве названий строк и столбцов используйте названия рецептов.

3.3 Какие рецепты являются наиболее похожими? Прокомментируйте результат (словами).